## Covid-19 Dashboard

Coursework Assignment: DIY Covid19 Dashboard 
ECS780P - Computer Programming - 2021/22

Since the beginning of the pandemic, statistics on hospital admissions and occupation of ventilator beds have been vital for understanding the level of spread of the Coronavirus and the severity of the illness. Below is a graph that shows the data, currently stored by the UK government, on how many hospital admissions there are on a daily basis (due to the Coronavirus) and how many mechnically ventilated beds are occupied daily. The data started being collected on slightly different days, but all around the time when the first lockdown occurred. *Exact dates are given below the graph.*

The graph will automatically show you the data for England. 

*Please use Ctrl and Click in order to view both hospital admissions and ventilator bed occupation at the same time.*

If you would like to compare this to Wales (or see the most up to date data on England), then please select the relevant 'API' button and wait until the button sign changes to a tick. Please see below the graph for specifics on when the data is collected and reported. 

In [2]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [3]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [5]:
with open("admisvsventbeds.json", "rt") as jsonfile:
    jsonwalesdata=json.load(jsonfile) #loads in the json file for Englands data that I have saved/submitted. 

In [24]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    dicentries=rawdata['data']
    dates=[entryrow['Date'] for entryrow in dicentries ]
    dates.sort()
    global startdate
    global enddate
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    templatedataframe=pd.DataFrame(index=index, columns=['Hospitaladmissions', 'Ventilatorbeds'])
    for row in dicentries:
        date=parse_date(row['Date'])
        for column in ['Hospitaladmissions', 'Ventilatorbeds']:
            if pd.isna(templatedataframe.loc[date, column]):  
                value= float(row[column]) if row[column]!=None else 0.0
                templatedataframe.loc[date, column]=value
    return(templatedataframe)
    
jsonwrangled=wrangle_data(jsonwalesdata)

In [15]:
def access_wales_api():
    filters = [
        "areaType=nation",
        "areaName=Wales"
] #using the areaName=Wales filter to allow for wales specific data

    structure = {
        "Date":"date",
        "Hospitaladmissions":"newAdmissions",
        "Ventilatorbeds":"covidOccupiedMVBeds"}

    api = Cov19API(filters=filters, structure=structure)
    data=api.get_json()
    return data

In [8]:
def access_england_api():
    filters = [
        "areaType=nation",
        "areaName=England"
]#using the areaName=England filter to allow for England specific data

    structure = {
        "Date":"date",
        "Hospitaladmissions":"newAdmissions",
        "Ventilatorbeds":"covidOccupiedMVBeds"}

    api = Cov19API(filters=filters, structure=structure)
    data=api.get_json()
    return data

In [28]:
def loc_graph(buttonvalues): #plots the graph and changes the description based on which option is selected
        jsonwrangled.plot(kind='line', y=list(buttonvalues))
        plt.show()
        print('The graph runs from ' + str(startdate) + ' to '+ str(enddate)) #shows dates covered in the graph and updates when the API is loaded in.
        description_of_options() #changes the descriptions along with when the API changes.
        
        
            
selection=wdg.SelectMultiple(
    options=['Hospitaladmissions', 'Ventilatorbeds'],
    value=['Hospitaladmissions'],
    rows=2,
    description='Data: ',
    disabled=False)



def refresh_graph():
    current=selection.value
    if selection.options[0] in current:
        other=(selection.options[1],)
    else:
        other=(selection.options[0],)
    selection.value=other
    selection.value=current
    
admissionsmetric="""Covid19 hospital admissions:
England:Reporting dates reflect admissions and new inpatient diagnoses for the previous day. 
Daily data at NHS Trust level is reported weekly on Thursdays.
Wales: Data are collected in a daily return from Digital Health and Care Wales (DHCW) 
and are reported daily Monday to Friday by StatsWales."""
ventilatormetric="""Currently occupied ventilator beds:
England: Data are collected daily and reported daily by NHS Trusts to NHS England and NHS Improvement. 
Daily data at NHS Trust level is reported weekly on Thursdays.
Wales: Data are collected in a daily return from Digital Health and Care Wales (DHCW) 
and are reported daily Monday to Friday by StatsWales."""
    
    
def description_of_options(): #wanted the description of how the data is reported for both hospital admissions 
    #and ventilator beds to be noted since I would want whoever reading to know that they are reported slightly differently. 
    "Retrieves a pre-written script related to which option is selected from the widget previously made"
    current=selection.value
    if selection.options[0]in current: 
        print(admissionsmetric)
    else:
        print(ventilatormetric)
    
    
    
graph=wdg.interactive_output(loc_graph, {'buttonvalues': selection})



layout=wdg.HBox([graph, selection])#changes the layout so that the widget and graph are side by side

In [10]:
def wales_api_button_callback(button):
    """ Button callback - accesses Wales specific API data, wrangles it and converts to a dataframe """
    apidata=access_wales_api()
    global jsonwrangled
    jsonwrangled=wrangle_data(apidata) 
    refresh_graph()
    apibutton.icon="check"
    #apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Wales API', 
    disabled=False,
    button_style='info', 
    tooltip="click to receive API data on Wales",
    icon='fa-paper-plane'
)

apibutton.on_click(wales_api_button_callback)

In [14]:
def england_api_button_callback(button):
    """ Button callback - accesses England specific API data, wrangles it and converts to a dataframe"""
    apidata=access_england_api()
    global jsonwrangled
    jsonwrangled=wrangle_data(apidata)
    refresh_graph()
    apibutton2.icon="check"
    #apibutton2.disabled=True

    
apibutton2=wdg.Button(
    description='England API', 
    disabled=False,
    button_style='info', 
    tooltip="click to receive API data on England",
    icon='fa-paper-plane'
)

apibutton2.on_click(england_api_button_callback)


In [27]:
buttonlayout=wdg.HBox([apibutton, apibutton2]) #puts the buttons next to each other. 
display(buttonlayout)


*Please note that the scale of the y-axis changes between England and Wales*

In [29]:
display(layout) #displays both the loc_graph function result and the widget

If you would like further information on the metrics used in these graphs, then please visit https://coronavirus.data.gov.uk/metrics and search the metric names 'newAdmissions' or 'covidoccupiedmvbeds'.

<br/>

When comparing the data from both, just based on the y-axis alone, you can see that the hospital admission rates and ventilator bed occupation is significantly lower in Wales than it is England. This could be down to difference in population size or how each country managed Covid restrictions. 

  ---

(C) **2021 Verity Spinks** (v.a.r.spinks@se21.qmul.ac.uk - web), all rights reserved.
Data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*
Coding reference: Fabrizio Smeraldi (f.smeraldi@qmul.ac.uk - web) from documents 'Accessing PHE Covid Data', 'Visualising the data', 'Adding interactive controls' and 'DIY Covid-19 Dashboard'. 